In [53]:
import pandas as pd
from collections import Counter
import numpy as np
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from linearmodels.iv import IV2SLS
import statsmodels.api as sm
from scipy.stats import norm
import statsmodels.formula.api as smf

In [54]:
# import pic data
all_pic_temp = pd.read_excel("data/input/open_pic_new.xlsx")
res_pic_temp = pd.read_excel("data/input/restaurant_new.xlsx")
drink_pic_temp = pd.read_excel("data/input/drink_new.xlsx")
# import business data 
business_feature_df = pd.read_csv("data/input/business_feature.csv")

# add photo count per business
photo_count = all_pic_temp.groupby('business_id').size().reset_index(name='photo_count')
business_feature_df = business_feature_df.merge(photo_count, on='business_id', how='left')
business_feature_df['photo_count'] = business_feature_df['photo_count'].fillna(0).astype(int)

In [55]:
print(round(all_pic_temp["memory_score"].mean(),3),all_pic_temp["memory_score"].min(),all_pic_temp["memory_score"].max())
print(round(res_pic_temp["memory_score"].mean(),3),res_pic_temp["memory_score"].min(),res_pic_temp["memory_score"].max())
print(round(drink_pic_temp["memory_score"].mean(),3),drink_pic_temp["memory_score"].min(),drink_pic_temp["memory_score"].max())

0.752 0.412 0.993
0.749 0.412 0.993
0.74 0.412 0.991


In [56]:
# data shape
print(all_pic_temp.shape,res_pic_temp.shape,drink_pic_temp.shape,business_feature_df.shape)

(164839, 31) (139385, 31) (73268, 33) (150346, 13)


In [57]:
def process_data(df_open):
  # filter yolo nan data
    var = []
    person_other_count = []
    person_total_count = []
    for index,row in df_open.iterrows():
        if len(str(row['objects_content']))==1 or len(str(row['objects_content']))==0:
            temp = ''
        else:
            # 'object_content' columns str to list
            temp = str(row['objects_content']).split(",")
            temp = [s.strip() for s in temp]
        person_count = temp.count('person')
        other_count = len(temp)-person_count
        temp_res = abs(person_count-other_count)
        try:
            var_temp = temp_res/(person_count+other_count)
        except:
            var_temp = 0
        var.append(var_temp)
        person_total_count.append(len(temp))
    df_open['var'] = var
    df_open['person_total_count'] = person_total_count
    df_open['person_total_count_cluster'] = np.where(df_open['person_total_count'] != 0, 1, df_open['person_total_count'])
    # print(df_open['person_total_count_cluster'].value_counts())
    filter_data  = df_open[df_open['person_total_count']!=0]
    print("pre process data len:",len(df_open),"post process data len: ",len(filter_data))
    return df_open[df_open['person_total_count']!=0]

In [58]:
# pic process
all_df_pic = process_data(all_pic_temp)
print("business",all_df_pic[['label','memory_score']].groupby('label').mean().reset_index())
res_df_pic = process_data(res_pic_temp)
print("res_df",res_df_pic[['label','memory_score']].groupby('label').mean().reset_index())
drink_df_pic = process_data(drink_pic_temp)
print("drink_df",drink_df_pic[['label','memory_score']].groupby('label').mean().reset_index())


pre process data len: 164839 post process data len:  154945
business      label  memory_score
0    drink      0.830912
1     food      0.788275
2   inside      0.668729
3     menu      0.826144
4  outside      0.690957
pre process data len: 139385 post process data len:  131624
res_df      label  memory_score
0    drink      0.826183
1     food      0.783535
2   inside      0.659904
3     menu      0.827708
4  outside      0.688542
pre process data len: 73268 post process data len:  69548
drink_df      label  memory_score
0    drink      0.831727
1     food      0.782448
2   inside      0.662736
3     menu      0.824719
4  outside      0.676933


In [59]:
def calculate_bubble_data(df, group_name):
    """计算气泡图需要的统计量"""
    stats = df.groupby('label')['memory_score'].agg(
        memory_score='mean',      # 平均值
        sd='std',                 # 标准差
        n='count'                 # 样本量
    ).reset_index()
   
    # 计算百分比
    total_n = stats['n'].sum()
    stats['percentage'] = (stats['n'] / total_n * 100)
    
    # 计算标准误和95% CI
    stats['se'] = stats['sd'] / np.sqrt(stats['n'])
    stats['ci_lower'] = stats['memory_score'] - 1.96 * stats['se']
    stats['ci_upper'] = stats['memory_score'] + 1.96 * stats['se']
    
    # 添加分组标签
    stats['group'] = group_name
    
    return stats

# ====================== 计算三组数据 ======================
all_business_stats = calculate_bubble_data(all_df_pic, "All businesses")
restaurant_stats = calculate_bubble_data(res_df_pic, "Restaurants")
drink_stats = calculate_bubble_data(drink_df_pic, "Drinks")

# ====================== 合并并保存 ======================
# 合并三个数据框
bubble_data = pd.concat([all_business_stats, restaurant_stats, drink_stats], ignore_index=True)

# 调整列顺序
bubble_data = bubble_data[['group', 'label', 'memory_score', 'sd', 'n', 'percentage', 'se', 'ci_lower', 'ci_upper']]

# # 查看数据
# print(bubble_data)

# 保存为CSV文件
bubble_data.to_excel("data/output/bubble_plot_data.xlsx", index=False)

print("\n文件已保存至: data/output/bubble_plot_data.xlsx")


文件已保存至: data/output/bubble_plot_data.xlsx


In [60]:
print(round(all_df_pic["memory_score"].mean(),3),all_df_pic["memory_score"].min(),all_df_pic["memory_score"].max())
print(round(res_df_pic["memory_score"].mean(),3),res_df_pic["memory_score"].min(),res_df_pic["memory_score"].max())
print(round(drink_df_pic["memory_score"].mean(),3),drink_df_pic["memory_score"].min(),drink_df_pic["memory_score"].max())

0.751 0.412 0.993
0.748 0.412 0.993
0.74 0.412 0.991


In [61]:
# business_feature data process
business_df =  business_feature_df[business_feature_df['business_id'].isin(list(all_df_pic['business_id'].unique()))]
business_res = business_feature_df[business_feature_df['business_id'].isin(list(res_df_pic['business_id'].unique()))]
business_drink = business_feature_df[business_feature_df['business_id'].isin(list(drink_df_pic['business_id'].unique()))]
# 22397 18203 8192

# pic 数据的转换
# print(len(business_df),len(business_res),len(business_drink))

business_df = pd.merge(business_df,all_df_pic[["business_id","memory_score","average_hue","average_saturation",
                                               "average_value","sharpness_measure","person_count","beauty_score",'person_total_count']].groupby("business_id").mean().reset_index(),
                       how='left',on='business_id')
business_res = pd.merge(business_res,res_df_pic[["business_id","memory_score","average_hue","average_saturation",
                                               "average_value","sharpness_measure","person_count","beauty_score",'person_total_count']].groupby("business_id").mean().reset_index(),
                       how='left',on='business_id')
business_drink = pd.merge(business_drink,drink_df_pic[["business_id","memory_score","average_hue","average_saturation",
                                               "average_value","sharpness_measure","person_count","beauty_score",'person_total_count']].groupby("business_id").mean().reset_index(),
                       how='left',on='business_id')
print(len(business_df),len(business_res),len(business_drink))

27075 21279 9853


In [62]:
business_df = business_df[(business_df['star_std']!=0)].reset_index(drop=True)
business_res = business_res[(business_res['star_std']!=0)].reset_index(drop=True)
business_drink = business_drink[(business_drink['star_std']!=0)].reset_index(drop=True)
print(len(business_df),len(business_res),len(business_drink))

# print(len(all_bus), len(all_res), len(all_drink))
# print(len(business_df), len(business_res), len(business_drink))
# print(len(all_bus)-len(business_df), len(all_res)-len(business_res), len(all_drink)-len(business_drink))
# print(round((len(all_bus)-len(business_df))/len(all_bus),3),
#        round((len(all_bus)-len(business_df))/len(all_res),3), 
#        round((len(all_bus)-len(business_df))/len(all_drink),3))

# 27075 21279 9853
# 26769 21150 9751
# 306 129 102
# 0.011 0.014 0.031

26769 21150 9751


In [63]:
print(business_df['review_count'].sum())
print(business_res['review_count'].sum())
print(business_drink['review_count'].sum())

3456545
3155433
1481608


# study 1 :image feature ---- memorability

In [64]:
# pic 均值和标准差的统计
def result_mean(data):
    res = [round(data['memory_score'].mean(),3),
    round(len(data[data['label']=='food'])/len(data),3),
    round(len(data[data['label']=='menu'])/len(data),3),
    round(len(data[data['label']=='inside'])/len(data),3),
    round(len(data[data['label']=='outside'])/len(data),3),
    round(len(data[data['label']=='drink'])/len(data),3),
    round(data['average_hue'].mean(),3),
    round(data['average_saturation'].mean(),3),
    round(data['average_value'].mean(),3),
    round(len(data[data['person_exist']==1])/len(data),3),
    # round(data['person_count'].mean(),3),
    round(data['var'].mean(),3),
    round(data['person_total_count'].mean(),3),
    round(data['sharpness_measure'].mean(),3),
    round(data['beauty_score'].mean(),3)
    ]
    return res

def result_std(data):
    res = [
    round(data['memory_score'].std(),3),
    '-','-','-','-','-',
    round(data['average_hue'].std(),3),
    round(data['average_saturation'].std(),3),
    round(data['average_value'].std(),3),
    '-',
    # round(data['person_count'].std(),3),
    round(data['var'].std(),3),
    round(data['person_total_count'].std(),3),
    round(data['sharpness_measure'].std(),3),
    round(data['beauty_score'].std(),3)
    ]
    return res

In [65]:
# pic 均值和标准差的统计结果
data = {
    "name":["memory_score","food","menu","inside",
            "outside","drink","average_hue","average_saturation",
            "average_value","person_exist","var","person_total_count","sharpness_measure","beauty_score"],
    'All Busi mean': result_mean(all_df_pic),
    'All Busi std': result_std(all_df_pic),
    'All Res mean': result_mean(res_df_pic),
    'All Res std': result_std(res_df_pic),
    'Drink mean': result_mean(drink_df_pic),
    'Drink std': result_std(drink_df_pic)
    }

# 创建DataFrame
df = pd.DataFrame(data)

# 显示DataFrame
print(df)
df.head()
df.to_excel("data/output/study1_picture_data_summary_yolo11.xlsx",index=False)
print("数据保存成功")

                  name  All Busi mean All Busi std  All Res mean All Res std  \
0         memory_score          0.751        0.106         0.748       0.105   
1                 food          0.547            -         0.591           -   
2                 menu          0.006            -         0.007           -   
3               inside          0.277            -         0.254           -   
4              outside          0.083            -         0.077           -   
5                drink          0.086            -         0.071           -   
6          average_hue         40.808       22.614        39.745      21.768   
7   average_saturation        101.648        39.34       103.231      38.646   
8        average_value        141.041       41.988       141.740      40.902   
9         person_exist          0.332            -         0.316           -   
10                 var          0.824        0.307         0.830       0.302   
11  person_total_count          5.605   

In [66]:
def create_ols_model(data):
    df_dummies = pd.get_dummies(data['label'])
    df_dummies = df_dummies[['food','menu','inside','drink']].astype(int)
    df = pd.concat([data, df_dummies], axis=1)
    # use StandardScaler
    scaler = StandardScaler()
    df['average_hue_s'] = scaler.fit_transform(df[['average_hue']])
    df['average_saturation_s'] = scaler.fit_transform(df[['average_saturation']])
    df['average_value_s'] = scaler.fit_transform(df[['average_value']])
    df['var'] = df[['var']]
    df['sharpness_measure_s'] = scaler.fit_transform(df[['sharpness_measure']])
    X =  df[list(df_dummies.columns)+['average_hue_s','average_saturation_s','average_value_s',
                                      'person_exist',
                                      'var',
                                      "person_total_count"
                                      ,'beauty_score','sharpness_measure_s'
                                      ]]
    y = data['memory_score']
    X = sm.add_constant(X)
    # Business-clustered SE to account for within-business dependence (Comment 1 fix)
    model = sm.OLS(y, X).fit(cov_type='cluster',
                              cov_kwds={'groups': data['business_id']})
    return model

In [67]:
# ols模型结果
print('--------------------------------------------------1.all--------------------------------------------------')
ols_business = create_ols_model(process_data(all_df_pic))
print(ols_business.summary())
print('--------------------------------------------------2.res_data--------------------------------------------------')
ols_res = create_ols_model(process_data(res_df_pic))
print(ols_res.summary())
print('--------------------------------------------------3.drink_data--------------------------------------------------')
ols_drink = create_ols_model(process_data(drink_df_pic))
print(ols_drink.summary())

--------------------------------------------------1.all--------------------------------------------------
pre process data len: 154945 post process data len:  154945
                            OLS Regression Results                            
Dep. Variable:           memory_score   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     4041.
Date:                Wed, 18 Feb 2026   Prob (F-statistic):               0.00
Time:                        15:41:26   Log-Likelihood:             1.6271e+05
No. Observations:              154945   AIC:                        -3.254e+05
Df Residuals:                  154932   BIC:                        -3.253e+05
Df Model:                          12                                         
Covariance Type:              cluster                                         
                           coef    std err  

In [68]:
# ======================== Study 1 OLS: Custom LaTeX with †/*/**/**** stars ========================

def _stars(p):
    if p < 0.001: return '***'
    if p < 0.01:  return '**'
    if p < 0.05:  return '*'
    if p < 0.10:  return '$^\\dagger$'
    return ''

def _fmt(val, decimals=4):
    return f"{val:.{decimals}f}"

models = [ols_business, ols_res, ols_drink]
col_names = ['All businesses', 'Restaurants', 'Drinks']

# Collect all variable names in display order
var_order = list(models[0].params.index)

# Build rows: coefficient with stars, then SE in parentheses
rows = []
for var in var_order:
    coef_cells = []
    se_cells = []
    for m in models:
        if var in m.params.index:
            coef_cells.append(f"{_fmt(m.params[var])}{_stars(m.pvalues[var])}")
            se_cells.append(f"({_fmt(m.bse[var])})")
        else:
            coef_cells.append('')
            se_cells.append('')
    rows.append((var, coef_cells, se_cells))

# Build LaTeX
lines = []
lines.append(r"\begin{table}[htbp]")
lines.append(r"\centering")
lines.append(r"\caption{OLS Regression Results: Image Memorability (Study 1)}")
lines.append(r"\label{tab:study1_ols}")
lines.append(r"\small")
ncols = len(models)
lines.append(r"\begin{tabular}{l" + " c" * ncols + "}")
lines.append(r"\toprule")
header = " & ".join([""] + col_names) + r" \\"
lines.append(header)
lines.append(r"\midrule")

for var, coef_cells, se_cells in rows:
    # Clean variable name for LaTeX
    var_tex = var.replace('_', r'\_')
    coef_row = " & ".join([var_tex] + coef_cells) + r" \\"
    se_row = " & ".join([""] + se_cells) + r" \\"
    lines.append(coef_row)
    lines.append(se_row)

lines.append(r"\midrule")
# N and R-squared
n_cells = [f"{int(m.nobs):,}" for m in models]
r2_cells = [_fmt(m.rsquared, 3) for m in models]
r2_adj_cells = [_fmt(m.rsquared_adj, 3) for m in models]
lines.append(" & ".join(["$N$"] + n_cells) + r" \\")
lines.append(" & ".join(["$R^2$"] + r2_cells) + r" \\")
lines.append(" & ".join(["Adj. $R^2$"] + r2_adj_cells) + r" \\")
lines.append(r"\bottomrule")
lines.append(r"\end{tabular}")
lines.append(r"\begin{minipage}{\textwidth}")
lines.append(r"\vspace{4pt}")
lines.append(r"\footnotesize")
lines.append(r"\textit{Note.} Standard errors clustered by business (in parentheses). "
             r"$^\dagger p < .10$, $^* p < .05$, $^{**} p < .01$, $^{***} p < .001$.")
lines.append(r"\end{minipage}")
lines.append(r"\end{table}")

table_tex = "\n".join(lines)
print(table_tex)

with open('data/output/study1_ols.tex', 'w', encoding='utf-8') as f:
    f.write(table_tex)
print("\nSaved: data/output/study1_ols.tex")


\begin{table}[htbp]
\centering
\caption{OLS Regression Results: Image Memorability (Study 1)}
\label{tab:study1_ols}
\small
\begin{tabular}{l c c c}
\toprule
 & All businesses & Restaurants & Drinks \\
\midrule
const & 0.6813*** & 0.6924*** & 0.6563*** \\
 & (0.0048) & (0.0052) & (0.0081) \\
food & 0.0854*** & 0.0826*** & 0.0907*** \\
 & (0.0015) & (0.0017) & (0.0022) \\
menu & 0.1191*** & 0.1207*** & 0.1355*** \\
 & (0.0031) & (0.0033) & (0.0041) \\
inside & -0.0007 & -0.0063** & 0.0061* \\
 & (0.0016) & (0.0019) & (0.0024) \\
drink & 0.1360*** & 0.1332*** & 0.1472*** \\
 & (0.0016) & (0.0018) & (0.0022) \\
average\_hue\_s & 0.0035*** & 0.0023*** & 0.0045*** \\
 & (0.0003) & (0.0004) & (0.0005) \\
average\_saturation\_s & -0.0026*** & -0.0018*** & -0.0025*** \\
 & (0.0003) & (0.0004) & (0.0004) \\
average\_value\_s & 0.0178*** & 0.0163*** & 0.0206*** \\
 & (0.0003) & (0.0004) & (0.0005) \\
person\_exist & 0.0050*** & 0.0018 & 0.0036$^\dagger$ \\
 & (0.0012) & (0.0015) & (0.0021) \\
va

# Study 1: Robustness Checks
Addressing 4 issues:
1. **1. Justify Clustering**: ICC ≈ 0.10–0.12 justifies business-clustered SE.
2. **2 (Bounded DV)**: OLS predictions checked within [0,1]; fractional logit robustness.
3. **3 Imbalance redundancy**: Conditional composition on people-present subsample.
4. **4 Circular hue**: Sin/cos hue parameterization with joint Wald test.

In [69]:
# ======================== Study 1 Robustness Checks ========================

# --- 1. Residual ICC to justify business-level clustering (ANOVA decomposition) ---
print("=== Residual ICC (ANOVA decomposition of OLS residuals) ===")

def compute_residual_icc(model, business_ids, sample_name):
    """
    ICC from OLS residuals grouped by business (one-way ANOVA decomposition).
    Measures within-business correlation after controlling for covariates.
    """
    resid = model.resid
    bid = business_ids.loc[resid.index]
    df_r = pd.DataFrame({'bid': bid.values, 'r': resid.values})
    groups = df_r.groupby('bid')['r']
    k = groups.ngroups
    n = len(df_r)
    gm = df_r['r'].mean()
    g_means = groups.mean()
    g_sizes = groups.size()
    SSB = ((g_means - gm)**2 * g_sizes).sum()
    SSW = groups.apply(lambda x: ((x - x.mean())**2).sum()).sum()
    MSB = SSB / (k - 1)
    MSW = SSW / (n - k)
    n0 = (n - (g_sizes**2).sum() / n) / (k - 1)
    icc = (MSB - MSW) / (MSB + (n0 - 1) * MSW)
    deff = 1 + (n0 - 1) * icc
    print(f"  {sample_name}: ICC = {icc:.4f}, DEFF = {deff:.2f} "
          f"(n={n:,}, clusters={k:,}, avg_cluster_size={n0:.1f})")
    return icc

icc_all = compute_residual_icc(ols_business, all_df_pic['business_id'], "All businesses")
icc_res = compute_residual_icc(ols_res, res_df_pic['business_id'], "Restaurants")
icc_drink = compute_residual_icc(ols_drink, drink_df_pic['business_id'], "Drinks")

# --- 2. OLS prediction bounds check (Comment 2: bounded DV) ---
print("\n=== OLS Prediction Bounds Check ===")
for name, model in [("All businesses", ols_business),
                     ("Restaurants", ols_res),
                     ("Drinks", ols_drink)]:
    pred = model.fittedvalues
    print(f"  {name}: min={pred.min():.4f}, max={pred.max():.4f}, "
          f"all in [0,1]: {bool((pred >= 0).all() and (pred <= 1).all())}")

# --- 3. Fractional logit robustness (Comment 2) ---
print("\n=== Fractional Logit Robustness (GLM Binomial, business-clustered SE) ===")

def create_fractional_logit(data):
    """Fractional logit (quasi-MLE) as robustness for bounded [0,1] DV."""
    df_dummies = pd.get_dummies(data['label'])
    df_dummies = df_dummies[['food','menu','inside','drink']].astype(int)
    df = pd.concat([data.copy(), df_dummies], axis=1)
    scaler = StandardScaler()
    df['average_hue_s'] = scaler.fit_transform(df[['average_hue']])
    df['average_saturation_s'] = scaler.fit_transform(df[['average_saturation']])
    df['average_value_s'] = scaler.fit_transform(df[['average_value']])
    df['sharpness_measure_s'] = scaler.fit_transform(df[['sharpness_measure']])
    X = df[list(df_dummies.columns)+['average_hue_s','average_saturation_s','average_value_s',
                                      'person_exist','var',"person_total_count",
                                      'beauty_score','sharpness_measure_s']]
    y = data['memory_score']
    X = sm.add_constant(X)
    model = sm.GLM(y, X, family=sm.families.Binomial()).fit(
        cov_type='cluster', cov_kwds={'groups': data['business_id']})
    return model

frac_all = create_fractional_logit(all_df_pic)
frac_res = create_fractional_logit(res_df_pic)
frac_drink = create_fractional_logit(drink_df_pic)

# Sign and significance comparison
print("\nSign/significance comparison (OLS vs Fractional Logit, All businesses):")
for v in ['food','menu','inside','drink','average_hue_s','average_saturation_s',
           'average_value_s','person_exist','var','person_total_count',
           'beauty_score','sharpness_measure_s']:
    o_s = '+' if ols_business.params[v] > 0 else '-'
    f_s = '+' if frac_all.params[v] > 0 else '-'
    match = "Y" if o_s == f_s else "N"
    print(f"  {v:25s}: OLS {o_s} (p={ols_business.pvalues[v]:.4f}) | "
          f"FracLogit {f_s} (p={frac_all.pvalues[v]:.4f}) | match: {match}")

frac_summary = summary_col([frac_all, frac_res, frac_drink],
    model_names=["Frac.Logit I", "Frac.Logit II", "Frac.Logit III"])
print("\n" + str(frac_summary))

# Full p-values for all three fractional logit models
print("\n--- Fractional Logit: Full coefficient table with p-values ---")
for name, model in [("All businesses", frac_all), ("Restaurants", frac_res), ("Drinks", frac_drink)]:
    print(f"\n  [{name}]")
    for v in model.params.index:
        print(f"    {v:25s}: coef={model.params[v]:10.6f}  SE={model.bse[v]:.6f}  p={model.pvalues[v]:.6f}")

# --- 4. Conditional composition robustness (Comment 3) ---
print("\n=== Conditional Composition Robustness (people-present subsample) ===")

def create_conditional_composition_model(data):
    """Clustered OLS on person_exist==1 subsample; var = conditional imbalance."""
    df = data[data['person_exist'] == 1].copy()
    df_dummies = pd.get_dummies(df['label'])
    for col in ['food','menu','inside','drink']:
        if col not in df_dummies.columns:
            df_dummies[col] = 0
    df_dummies = df_dummies[['food','menu','inside','drink']].astype(int)
    df = pd.concat([df, df_dummies], axis=1)
    scaler = StandardScaler()
    df['average_hue_s'] = scaler.fit_transform(df[['average_hue']])
    df['average_saturation_s'] = scaler.fit_transform(df[['average_saturation']])
    df['average_value_s'] = scaler.fit_transform(df[['average_value']])
    df['sharpness_measure_s'] = scaler.fit_transform(df[['sharpness_measure']])
    # person_exist dropped (constant=1 in this subsample)
    X = df[list(df_dummies.columns)+['average_hue_s','average_saturation_s','average_value_s',
                                      'var',"person_total_count",
                                      'beauty_score','sharpness_measure_s']]
    y = df['memory_score']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df['business_id']})
    return model, len(df), df['business_id'].nunique()

cond_all, n_cond_all, nc_all = create_conditional_composition_model(all_df_pic)
cond_res, n_cond_res, nc_res = create_conditional_composition_model(res_df_pic)
cond_drink, n_cond_drink, nc_drink = create_conditional_composition_model(drink_df_pic)

print("Baseline (full) vs Conditional (people-present) composition coefficient:")
for name, base, cond, n_c, nc in [
    ("All businesses", ols_business, cond_all, n_cond_all, nc_all),
    ("Restaurants", ols_res, cond_res, n_cond_res, nc_res),
    ("Drinks", ols_drink, cond_drink, n_cond_drink, nc_drink)]:
    print(f"  {name}: baseline var={base.params['var']:.4f} (p={base.pvalues['var']:.4g}) | "
          f"conditional var={cond.params['var']:.4f} (p={cond.pvalues['var']:.4g}) | "
          f"N={n_c:,}, clusters={nc:,}")

cond_summary = summary_col([cond_all, cond_res, cond_drink],
    model_names=["Cond.Comp I", "Cond.Comp II", "Cond.Comp III"])
print("\n" + str(cond_summary))

# Full p-values for all three conditional composition models
print("\n--- Conditional Composition: Full coefficient table with p-values ---")
for name, model in [("All businesses", cond_all), ("Restaurants", cond_res), ("Drinks", cond_drink)]:
    print(f"\n  [{name}] (N={int(model.nobs):,})")
    for v in model.params.index:
        print(f"    {v:25s}: coef={model.params[v]:10.6f}  SE={model.bse[v]:.6f}  p={model.pvalues[v]:.6f}")

# --- 5. Circular hue robustness (Comment 4) ---
print("\n=== Circular Hue Robustness (sin/cos parameterization, clustered SE) ===")

def create_circular_hue_model(data):
    """OLS with sin/cos hue replacing linear hue, business-clustered SE."""
    df_dummies = pd.get_dummies(data['label'])
    df_dummies = df_dummies[['food','menu','inside','drink']].astype(int)
    df = pd.concat([data.copy(), df_dummies], axis=1)
    scaler = StandardScaler()
    theta = 2.0 * np.pi * data['average_hue'].values / 180.0  # OpenCV hue period=180
    df['hue_sin'] = np.sin(theta)
    df['hue_cos'] = np.cos(theta)
    df['average_saturation_s'] = scaler.fit_transform(df[['average_saturation']])
    df['average_value_s'] = scaler.fit_transform(df[['average_value']])
    df['sharpness_measure_s'] = scaler.fit_transform(df[['sharpness_measure']])
    X = df[list(df_dummies.columns)+['hue_sin','hue_cos','average_saturation_s',
                                      'average_value_s','person_exist','var',
                                      "person_total_count",'beauty_score',
                                      'sharpness_measure_s']]
    y = data['memory_score']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data['business_id']})
    wald = model.wald_test("hue_sin = 0, hue_cos = 0", scalar=True)
    return model, wald

circ_all, wald_all = create_circular_hue_model(all_df_pic)
circ_res, wald_res = create_circular_hue_model(res_df_pic)
circ_drink, wald_drink = create_circular_hue_model(drink_df_pic)

print("Linear vs Circular hue:")
for name, base, circ, wald in [
    ("All businesses", ols_business, circ_all, wald_all),
    ("Restaurants", ols_res, circ_res, wald_res),
    ("Drinks", ols_drink, circ_drink, wald_drink)]:
    print(f"  {name}: linear hue beta={base.params['average_hue_s']:.4f} "
          f"(p={base.pvalues['average_hue_s']:.4g}) | "
          f"circular Wald p={float(wald.pvalue):.4g} | "
          f"sin={circ.params['hue_sin']:.4f}, cos={circ.params['hue_cos']:.4f}")

circ_summary = summary_col([circ_all, circ_res, circ_drink],
    model_names=["Circ.Hue I", "Circ.Hue II", "Circ.Hue III"])
print("\n" + str(circ_summary))

# Full p-values for all three circular hue models
print("\n--- Circular Hue: Full coefficient table with p-values ---")
for name, model, wald in [("All businesses", circ_all, wald_all),
                            ("Restaurants", circ_res, wald_res),
                            ("Drinks", circ_drink, wald_drink)]:
    print(f"\n  [{name}] Joint Wald F={float(wald.statistic):.4f}, p={float(wald.pvalue):.4g}")
    for v in model.params.index:
        print(f"    {v:25s}: coef={model.params[v]:10.6f}  SE={model.bse[v]:.6f}  p={model.pvalues[v]:.6f}")

# --- 5b. Hue origin-rotation sensitivity analysis ---
print("\n=== Hue Origin-Rotation Sensitivity ===")
print("Shifting the hue origin by 0-150 degrees and re-fitting linear hue OLS.")
print("If the linear coefficient sign flips, it confirms the direction is origin-dependent.\n")

shifts = [0, 30, 60, 90, 120, 150]

def fit_rotated_linear_hue(data, shift_deg):
    """Re-fit clustered OLS with hue origin shifted by shift_deg (mod 180)."""
    df_dummies = pd.get_dummies(data['label'])
    df_dummies = df_dummies[['food','menu','inside','drink']].astype(int)
    df = pd.concat([data.copy(), df_dummies], axis=1)
    scaler = StandardScaler()
    df['average_hue_rot'] = (data['average_hue'] - shift_deg) % 180.0
    df['average_hue_rot_s'] = scaler.fit_transform(df[['average_hue_rot']])
    df['average_saturation_s'] = scaler.fit_transform(df[['average_saturation']])
    df['average_value_s'] = scaler.fit_transform(df[['average_value']])
    df['sharpness_measure_s'] = scaler.fit_transform(df[['sharpness_measure']])
    X = df[list(df_dummies.columns)+['average_hue_rot_s','average_saturation_s',
                                      'average_value_s','person_exist','var',
                                      "person_total_count",'beauty_score',
                                      'sharpness_measure_s']]
    y = data['memory_score']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data['business_id']})
    return model

for name, data in [("All businesses", all_df_pic),
                     ("Restaurants", res_df_pic),
                     ("Drinks", drink_df_pic)]:
    print(f"  [{name}]")
    print(f"    {'Shift':>6s}  {'Hue coef':>10s}  {'SE':>10s}  {'p-value':>10s}  {'Sign':>5s}")
    betas = []
    for s in shifts:
        m = fit_rotated_linear_hue(data, s)
        beta = m.params['average_hue_rot_s']
        se = m.bse['average_hue_rot_s']
        pv = m.pvalues['average_hue_rot_s']
        betas.append(beta)
        sign = '+' if beta > 0 else '-'
        print(f"    {s:>5d}deg  {beta:>10.6f}  {se:>10.6f}  {pv:>10.6f}  {sign:>5s}")
    sign_changes = sum(1 for i in range(len(betas)-1)
                       if (betas[i] > 0) != (betas[i+1] > 0))
    print(f"    Range: [{min(betas):.6f}, {max(betas):.6f}], sign changes: {sign_changes}")
    print()

# --- Save robustness LaTeX tables (custom stars: dagger/*/**/***) ---

def _stars(p):
    if p < 0.001: return '***'
    if p < 0.01:  return '**'
    if p < 0.05:  return '*'
    if p < 0.10:  return '$^\dagger$'
    return ''

def _fmt(val, decimals=4):
    return f"{val:.{decimals}f}"

def save_custom_robustness_tex(models, col_names, filename, caption, label):
    """Build LaTeX table from model list with custom star convention."""
    BS = chr(92) + chr(92)  # LaTeX row ending: two backslashes
    var_order = list(models[0].params.index)
    lines = []
    lines.append(r'\begin{table}[htbp]')
    lines.append(r'\centering')
    lines.append(f'\caption{{{caption}}}')
    lines.append(f'\label{{{label}}}')
    lines.append(r'\small')
    lines.append(r'\begin{tabular}{l' + ' c' * len(models) + '}')
    lines.append(r'\toprule')
    lines.append(' & '.join([''] + col_names) + ' ' + BS)
    lines.append(r'\midrule')
    for var in var_order:
        coef_cells, se_cells = [], []
        for m in models:
            if var in m.params.index:
                coef_cells.append(f'{_fmt(m.params[var])}{_stars(m.pvalues[var])}')
                se_cells.append(f'({_fmt(m.bse[var])})')
            else:
                coef_cells.append('')
                se_cells.append('')
        var_tex = var.replace('_', r'\_')
        lines.append(' & '.join([var_tex] + coef_cells) + ' ' + BS)
        lines.append(' & '.join([''] + se_cells) + ' ' + BS)
    lines.append(r'\midrule')
    n_cells = [f'{int(m.nobs):,}' for m in models]
    lines.append(' & '.join([r'$N$'] + n_cells) + ' ' + BS)
    if hasattr(models[0], 'rsquared'):
        r2 = [_fmt(m.rsquared, 3) if hasattr(m, 'rsquared') else '' for m in models]
        lines.append(' & '.join([r'$R^2$'] + r2) + ' ' + BS)
    lines.append(r'\bottomrule')
    lines.append(r'\end{tabular}')
    lines.append(r'\begin{minipage}{\textwidth}')
    lines.append(r'\vspace{4pt}')
    lines.append(r'\footnotesize')
    lines.append(r'\textit{Note.} Standard errors clustered by business (in parentheses). '
                 r'$^\dagger p < .10$, $^* p < .05$, $^{**} p < .01$, $^{***} p < .001$.')
    lines.append(r'\end{minipage}')
    lines.append(r'\end{table}')
    table_tex = '\n'.join(lines)
    with open(f'data/output/{filename}', 'w', encoding='utf-8') as f:
        f.write(table_tex)
    print(f'Saved: data/output/{filename}')

save_custom_robustness_tex(
    [frac_all, frac_res, frac_drink],
    ['All businesses', 'Restaurants', 'Drinks'],
    'study1_fractional_logit_cluster.tex',
    'Fractional Logit Robustness Check (Study 1)',
    'tab:frac_logit')

save_custom_robustness_tex(
    [cond_all, cond_res, cond_drink],
    ['All businesses', 'Restaurants', 'Drinks'],
    'study1_conditional_composition_cluster.tex',
    'Conditional Composition Robustness: People-Present Subsample (Study 1)',
    'tab:cond_comp')

save_custom_robustness_tex(
    [circ_all, circ_res, circ_drink],
    ['All businesses', 'Restaurants', 'Drinks'],
    'study1_circular_hue_cluster.tex',
    'Circular Hue Decomposition Robustness Check (Study 1)',
    'tab:circ_hue')

print('\nAll robustness checks complete.')


=== Residual ICC (ANOVA decomposition of OLS residuals) ===


<>:246: SyntaxWarning: invalid escape sequence '\d'
<>:259: SyntaxWarning: invalid escape sequence '\c'
<>:260: SyntaxWarning: invalid escape sequence '\l'
<>:246: SyntaxWarning: invalid escape sequence '\d'
<>:259: SyntaxWarning: invalid escape sequence '\c'
<>:260: SyntaxWarning: invalid escape sequence '\l'
C:\Users\xuyuhong\AppData\Local\Temp\ipykernel_75828\290770572.py:246: SyntaxWarning: invalid escape sequence '\d'
  if p < 0.10:  return '$^\dagger$'
C:\Users\xuyuhong\AppData\Local\Temp\ipykernel_75828\290770572.py:259: SyntaxWarning: invalid escape sequence '\c'
  lines.append(f'\caption{{{caption}}}')
C:\Users\xuyuhong\AppData\Local\Temp\ipykernel_75828\290770572.py:260: SyntaxWarning: invalid escape sequence '\l'
  lines.append(f'\label{{{label}}}')


  All businesses: ICC = 0.1186, DEFF = 1.56 (n=154,945, clusters=27,075, avg_cluster_size=5.7)
  Restaurants: ICC = 0.0979, DEFF = 1.51 (n=131,624, clusters=21,279, avg_cluster_size=6.2)
  Drinks: ICC = 0.0983, DEFF = 1.60 (n=69,548, clusters=9,853, avg_cluster_size=7.1)

=== OLS Prediction Bounds Check ===
  All businesses: min=0.4623, max=0.9004, all in [0,1]: True
  Restaurants: min=0.4607, max=0.8927, all in [0,1]: True
  Drinks: min=0.4643, max=0.9080, all in [0,1]: True

=== Fractional Logit Robustness (GLM Binomial, business-clustered SE) ===

Sign/significance comparison (OLS vs Fractional Logit, All businesses):
  food                     : OLS + (p=0.0000) | FracLogit + (p=0.0000) | match: Y
  menu                     : OLS + (p=0.0000) | FracLogit + (p=0.0000) | match: Y
  inside                   : OLS - (p=0.6503) | FracLogit + (p=0.1567) | match: N
  drink                    : OLS + (p=0.0000) | FracLogit + (p=0.0000) | match: Y
  average_hue_s            : OLS + (p=0.000

In [70]:
# ======================== People's Presence: Paradox & "Others" Category ========================

# --- 6a. Create "Others" subsample (non-restaurant, non-drink businesses) ---
other_bids = set(all_df_pic['business_id'].unique()) - set(res_df_pic['business_id'].unique()) - set(drink_df_pic['business_id'].unique())
other_df_pic = all_df_pic[all_df_pic['business_id'].isin(other_bids)].copy()
print(f"Others subsample: {len(other_df_pic):,} images, {len(other_bids):,} businesses\n")

# --- 6b. Crosstab: person_exist rate by content type ---
print("=== Person Presence Rate by Content Type ===\n")

for name, data in [("All businesses", all_df_pic),
                     ("Restaurants", res_df_pic),
                     ("Drinks", drink_df_pic),
                     ("Others (non-rest, non-drink)", other_df_pic)]:
    ct = data.groupby('label')['person_exist'].agg(['mean', 'sum', 'count'])
    ct.columns = ['person_rate', 'n_with_person', 'n_total']
    ct['pct_of_sample'] = (ct['n_total'] / ct['n_total'].sum() * 100).round(1)
    ct['person_rate'] = (ct['person_rate'] * 100).round(1)
    print(f"  [{name}] (N={len(data):,})")
    print(f"    {'Label':>10s}  {'% of sample':>12s}  {'person_exist %':>15s}  {'n_with_person':>14s}")
    for label, row in ct.iterrows():
        print(f"    {label:>10s}  {row['pct_of_sample']:>11.1f}%  {row['person_rate']:>14.1f}%  {int(row['n_with_person']):>14,}")
    # Overall person_exist rate
    overall_rate = data['person_exist'].mean() * 100
    print(f"    {'OVERALL':>10s}  {'100.0':>11s}%  {overall_rate:>14.1f}%  {int(data['person_exist'].sum()):>14,}")
    print()

# --- 6c. Nested models: person_exist BEFORE vs AFTER content controls ---
print("=== Nested Models: Person Presence Before/After Content Controls ===\n")
print("  Model A = color + sharpness + aesthetics + person_exist + var + total_objects (NO content dummies)")
print("  Model B = Model A + content dummies (food, menu, inside, drink)")
print("  if coefficient FLIPS SIGN between A and B, person_exist is confounded with content type.\n")

def fit_nested_models(data, sample_name):
    """
    Model A: WITHOUT content dummies → captures total (confounded) effect
    Model B: WITH content dummies    → captures conditional (partial) effect
    Sign reversal: person_exist covaries with a content type
    that has opposite memorability association.
    """
    df = data.copy()
    df_dummies = pd.get_dummies(df['label'])
    for col in ['food','menu','inside','drink']:
        if col not in df_dummies.columns:
            df_dummies[col] = 0
    df_dummies = df_dummies[['food','menu','inside','drink']].astype(int)
    df = pd.concat([df, df_dummies], axis=1)
    scaler = StandardScaler()
    df['average_hue_s'] = scaler.fit_transform(df[['average_hue']])
    df['average_saturation_s'] = scaler.fit_transform(df[['average_saturation']])
    df['average_value_s'] = scaler.fit_transform(df[['average_value']])
    df['sharpness_measure_s'] = scaler.fit_transform(df[['sharpness_measure']])
    y = df['memory_score']
    groups = df['business_id']

    # Model A: WITHOUT content dummies
    X_a = df[['average_hue_s','average_saturation_s','average_value_s',
              'person_exist','var','person_total_count',
              'beauty_score','sharpness_measure_s']]
    X_a = sm.add_constant(X_a)
    model_a = sm.OLS(y, X_a).fit(cov_type='cluster', cov_kwds={'groups': groups})

    # Model B: WITH content dummies (same as main model)
    X_b = df[['food','menu','inside','drink',
              'average_hue_s','average_saturation_s','average_value_s',
              'person_exist','var','person_total_count',
              'beauty_score','sharpness_measure_s']]
    X_b = sm.add_constant(X_b)
    model_b = sm.OLS(y, X_b).fit(cov_type='cluster', cov_kwds={'groups': groups})

    # Extract person_exist results
    pe_a = {'coef': model_a.params['person_exist'], 'se': model_a.bse['person_exist'],
            'p': model_a.pvalues['person_exist'], 'ci_lo': model_a.conf_int().loc['person_exist', 0],
            'ci_hi': model_a.conf_int().loc['person_exist', 1]}
    pe_b = {'coef': model_b.params['person_exist'], 'se': model_b.bse['person_exist'],
            'p': model_b.pvalues['person_exist'], 'ci_lo': model_b.conf_int().loc['person_exist', 0],
            'ci_hi': model_b.conf_int().loc['person_exist', 1]}

    sign_a = '+' if pe_a['coef'] > 0 else '-'
    sign_b = '+' if pe_b['coef'] > 0 else '-'
    reversal = "SIGN REVERSAL" if sign_a != sign_b else "same sign"

    print(f"  [{sample_name}] (N={int(model_a.nobs):,})")
    print(f"    Model A (no content):   person_exist = {pe_a['coef']:+.6f} "
          f"(SE={pe_a['se']:.6f}, p={pe_a['p']:.4f}, 95%CI=[{pe_a['ci_lo']:.6f}, {pe_a['ci_hi']:.6f}])")
    print(f"    Model B (with content): person_exist = {pe_b['coef']:+.6f} "
          f"(SE={pe_b['se']:.6f}, p={pe_b['p']:.4f}, 95%CI=[{pe_b['ci_lo']:.6f}, {pe_b['ci_hi']:.6f}])")
    print(f"    >>> {reversal}: {sign_a} -> {sign_b}, "
          f"delta = {pe_b['coef'] - pe_a['coef']:+.6f}")
    print()
    return model_a, model_b

# Run for all four samples
nested_a_all, nested_b_all = fit_nested_models(all_df_pic, "All businesses")
nested_a_res, nested_b_res = fit_nested_models(res_df_pic, "Restaurants")
nested_a_drink, nested_b_drink = fit_nested_models(drink_df_pic, "Drinks")
nested_a_other, nested_b_other = fit_nested_models(other_df_pic, "Others (non-rest, non-drink)")

# --- 6d. Main model 95% CIs for person_exist ---
print("=== 95% CI for person_exist in main clustered OLS ===")
for name, model in [("All businesses", ols_business),
                     ("Restaurants", ols_res),
                     ("Drinks", ols_drink)]:
    ci = model.conf_int().loc['person_exist']
    print(f"  {name}: beta={model.params['person_exist']:.4f}, "
          f"95% CI = [{ci[0]:.4f}, {ci[1]:.4f}], p={model.pvalues['person_exist']:.4f}")

# --- 6e. Content-type memorability: are inside shots less memorable? ---
print("\n=== Average Memorability by Content Type ===\n")
for name, data in [("All businesses", all_df_pic),
                     ("Restaurants", res_df_pic),
                     ("Drinks", drink_df_pic),
                     ("Others", other_df_pic)]:
    ct = data.groupby('label')['memory_score'].agg(['mean', 'std', 'count'])
    print(f"  [{name}]")
    for label, row in ct.iterrows():
        print(f"    {label:>10s}: mean={row['mean']:.4f}, sd={row['std']:.4f}, n={int(row['count']):,}")
    print()


Others subsample: 10,331 images, 2,969 businesses

=== Person Presence Rate by Content Type ===

  [All businesses] (N=154,945)
         Label   % of sample   person_exist %   n_with_person
         drink          8.6%            25.0%           3,335
          food         54.7%            17.0%          14,406
        inside         27.7%            63.0%          27,070
          menu          0.6%            16.0%             160
       outside          8.3%            50.7%           6,534
       OVERALL        100.0%            33.2%          51,505

  [Restaurants] (N=131,624)
         Label   % of sample   person_exist %   n_with_person
         drink          7.1%            26.6%           2,501
          food         59.1%            16.4%          12,781
        inside         25.4%            63.1%          21,087
          menu          0.7%            16.0%             142
       outside          7.7%            50.0%           5,085
       OVERALL        100.0%         

In [71]:
# ======================== Supplementary LaTeX Tables for study 1 results ========================

# ---------- Person Presence & Memorability by Content Type ----------

def build_panel_a(data, panel_label):
    """Build rows for one panel (one subsample)."""
    rows = []
    ct = data.groupby('label').agg(
        n_total=('memory_score', 'count'),
        person_rate=('person_exist', 'mean'),
        mem_mean=('memory_score', 'mean'),
        mem_sd=('memory_score', 'std')
    )
    total_n = ct['n_total'].sum()
    for label in ['food', 'inside', 'drink', 'outside', 'menu']:
        if label not in ct.index:
            continue
        r = ct.loc[label]
        pct_sample = r['n_total'] / total_n * 100
        rows.append(
            f"  {label.capitalize():10s} & {int(r['n_total']):,} & {pct_sample:.1f}\\% "
            f"& {r['person_rate']*100:.1f}\\% & {r['mem_mean']:.3f} & ({r['mem_sd']:.3f}) \\\\"
        )
    # Overall row
    overall_person = data['person_exist'].mean() * 100
    overall_mem = data['memory_score'].mean()
    overall_sd = data['memory_score'].std()
    rows.append(f"  \\midrule")
    rows.append(
        f"  Overall    & {len(data):,} & 100.0\\% "
        f"& {overall_person:.1f}\\% & {overall_mem:.3f} & ({overall_sd:.3f}) \\\\"
    )
    header = f"\\midrule\n\\multicolumn{{6}}{{l}}{{\\textit{{Panel {panel_label}}}}} \\\\\n\\midrule"
    return header + "\n" + "\n".join(rows)

panels_a = {
    'A: All businesses': all_df_pic,
    'B: Restaurants': res_df_pic,
    'C: Drink-related': drink_df_pic,
    'D: Other businesses': other_df_pic,
}

table_a_body = "\n".join(build_panel_a(data, label) for label, data in panels_a.items())

table_a = r"""\begin{table}[htbp]
\centering
\caption{Person Presence Rates and Average Memorability by Image Content Type}
\label{tab:person_content_crosstab}
\small
\begin{tabular}{l r r r r r}
\toprule
Content type & $N$ & \% of sample & Person present (\%) & Mean memorability & (SD) \\
""" + table_a_body + r"""
\bottomrule
\end{tabular}
\begin{minipage}{\textwidth}
\vspace{4pt}
\footnotesize
\textit{Note.} Each panel reports one subsample. ``Person present'' is the percentage of images
in that content category containing at least one YOLO-detected person. ``Other businesses'' are establishments outside
the restaurant and drink categories (retail, services, entertainment, etc.). People appear
disproportionately in interior shots across all subsamples, and inside images
carry the lowest average memorability (sede figure 2).
\end{minipage}
\end{table}"""

print(table_a)

with open('data/output/supplementary_table_person_content.tex', 'w', encoding='utf-8') as f:
    f.write(table_a)
print("\nSaved: data/output/supplementary_table_person_content.tex")

# ---------- Nested Model Comparison  ----------

def build_nested_row(data, sample_label):
    """Fit Model A (no content) and Model B (with content), return LaTeX row."""
    df = data.copy()
    df_dummies = pd.get_dummies(df['label'])
    for col in ['food','menu','inside','drink']:
        if col not in df_dummies.columns:
            df_dummies[col] = 0
    df_dummies = df_dummies[['food','menu','inside','drink']].astype(int)
    df = pd.concat([df, df_dummies], axis=1)
    scaler = StandardScaler()
    df['average_hue_s'] = scaler.fit_transform(df[['average_hue']])
    df['average_saturation_s'] = scaler.fit_transform(df[['average_saturation']])
    df['average_value_s'] = scaler.fit_transform(df[['average_value']])
    df['sharpness_measure_s'] = scaler.fit_transform(df[['sharpness_measure']])
    y = df['memory_score']
    groups = df['business_id']

    # Model A: no content dummies
    X_a = sm.add_constant(df[['average_hue_s','average_saturation_s','average_value_s',
              'person_exist','var','person_total_count','beauty_score','sharpness_measure_s']])
    model_a = sm.OLS(y, X_a).fit(cov_type='cluster', cov_kwds={'groups': groups})

    # Model B: with content dummies
    X_b = sm.add_constant(df[['food','menu','inside','drink',
              'average_hue_s','average_saturation_s','average_value_s',
              'person_exist','var','person_total_count','beauty_score','sharpness_measure_s']])
    model_b = sm.OLS(y, X_b).fit(cov_type='cluster', cov_kwds={'groups': groups})

    a_coef = model_a.params['person_exist']
    a_se = model_a.bse['person_exist']
    a_p = model_a.pvalues['person_exist']
    b_coef = model_b.params['person_exist']
    b_se = model_b.bse['person_exist']
    b_p = model_b.pvalues['person_exist']
    delta = b_coef - a_coef

    def stars(p):
        if p < 0.001: return '***'
        if p < 0.01: return '**'
        if p < 0.05: return '*'
        if p < 0.10: return '$^\\dagger$'
        return ''

    n_obs = int(model_a.nobs)
    n_clust = df['business_id'].nunique()

    row = (f"  {sample_label} & {n_obs:,} & {n_clust:,} "
           f"& {a_coef:.4f}{stars(a_p)} & ({a_se:.4f}) "
           f"& {b_coef:.4f}{stars(b_p)} & ({b_se:.4f}) "
           f"& {delta:+.4f} \\\\")
    return row

nested_rows = []
for label, data in [("All businesses", all_df_pic),
                     ("Restaurants", res_df_pic),
                     ("Drink-related", drink_df_pic),
                     ("Other businesses", other_df_pic)]:
    nested_rows.append(build_nested_row(data, label))

table_b = r"""\begin{table}[htbp]
\centering
\caption{Person Presence--Memorability Relationship: Nested Model Comparison}
\label{tab:nested_person}
\small
\begin{tabular}{l r r r r r r r}
\toprule
 & & & \multicolumn{2}{c}{Model A} & \multicolumn{2}{c}{Model B} & \\
\cmidrule(lr){4-5} \cmidrule(lr){6-7}
Sample & $N$ & Clusters & $\hat{\beta}$ & (SE) & $\hat{\beta}$ & (SE) & $\Delta\hat{\beta}$ \\
\midrule
""" + "\n".join(nested_rows) + r"""
\bottomrule
\end{tabular}
\begin{minipage}{\textwidth}
\vspace{4pt}
\footnotesize
\textit{Note.} Model~A regresses image memorability on colour properties (hue, saturation,
brightness), sharpness, aesthetic quality, person presence, people--object imbalance,
and total object count, \textit{without} content-category indicators. Model~B adds four
content-type dummies (food, menu, inside, drink; outside is the reference category).
Both models use OLS with standard errors clustered at the business level.
$\Delta\hat{\beta} = \hat{\beta}_B - \hat{\beta}_A$ captures the change in the
person-presence coefficient when content type is controlled. The sign reversal from
negative (Model~A) to positive (Model~B) across all four subsamples constitutes a paradox: images containing people are disproportionately interior shots,
which have lower memorability, so the unconditional association is negative. Once
content type is controlled, person presence is associated with modestly
\textit{higher} memorability. The conditional effect is largest for non-restaurant,
non-drink businesses, where human figures are less routine and more visually distinctive.
$^\dagger p < .10$, $^* p < .05$, $^{**} p < .01$, $^{***} p < .001$.
\end{minipage}
\end{table}"""

print("\n" + table_b)

with open('data/output/supplementary_table_nested_person.tex', 'w', encoding='utf-8') as f:
    f.write(table_b)
print("\nSaved: data/output/supplementary_table_nested_person.tex")


\begin{table}[htbp]
\centering
\caption{Person Presence Rates and Average Memorability by Image Content Type}
\label{tab:person_content_crosstab}
\small
\begin{tabular}{l r r r r r}
\toprule
Content type & $N$ & \% of sample & Person present (\%) & Mean memorability & (SD) \\
\midrule
\multicolumn{6}{l}{\textit{Panel A: All businesses}} \\
\midrule
  Food       & 84,788 & 54.7\% & 17.0\% & 0.788 & (0.082) \\
  Inside     & 42,935 & 27.7\% & 63.0\% & 0.669 & (0.100) \\
  Drink      & 13,338 & 8.6\% & 25.0\% & 0.831 & (0.072) \\
  Outside    & 12,885 & 8.3\% & 50.7\% & 0.691 & (0.103) \\
  Menu       & 999 & 0.6\% & 16.0\% & 0.826 & (0.069) \\
  \midrule
  Overall    & 154,945 & 100.0\% & 33.2\% & 0.751 & (0.106) \\
\midrule
\multicolumn{6}{l}{\textit{Panel B: Restaurants}} \\
\midrule
  Food       & 77,784 & 59.1\% & 16.4\% & 0.784 & (0.081) \\
  Inside     & 33,395 & 25.4\% & 63.1\% & 0.660 & (0.097) \\
  Drink      & 9,397 & 7.1\% & 26.6\% & 0.826 & (0.074) \\
  Outside    & 10,162 & 

# study 2: Memorability- business ratings

In [72]:
# data process
def contains_one(lst):
    return 1 if 1 in lst else 0


def bus_pic_data_process(business_df,pic_df):
    scaler = StandardScaler()
    business_id_unique = business_df['business_id'].unique().tolist()
    count = 0
    pic_bus_df = []
    for business_id in business_id_unique[:]:
        count+=1
        single_bus = pic_df[pic_df['business_id']==business_id]
        food_rate = round(len(single_bus[single_bus['label']=='food'])/len(single_bus),3)
        drink_rate = round(len(single_bus[single_bus['label']=='drink'])/len(single_bus),3)
        menu_rate = round(len(single_bus[single_bus['label']=='menu'])/len(single_bus),3)
        inside_rate = round(len(single_bus[single_bus['label']=='inside'])/len(single_bus),3)
        outside_rate = round(len(single_bus[single_bus['label']=='outside'])/len(single_bus),3)
        person_exist = contains_one(single_bus['person_exist'].values.tolist())
        # person percentage
        try:
            person_percentage = single_bus['person_exist'].sum()/len(single_bus)
        except:
            person_percentage = 0
        avg_var = round(single_bus['var'].mean(),3)
        person_total_count = round(single_bus['person_total_count'].mean(),3)
        person_count = round(single_bus['person_count'].mean(),3)
        pic_bus_df.append([business_id,food_rate,drink_rate,menu_rate,inside_rate,outside_rate,avg_var,person_exist,person_percentage,person_total_count, person_count])
    pic_bus_df = pd.DataFrame(pic_bus_df,columns=['business_id','food','drink','menu','inside','outside','var','person_exist',"person_percentage",'person_total_count','person_count'])
    bus_data = business_df
    # # data merge
    res = pd.merge(bus_data,pic_bus_df,on='business_id')
    return res
business_df = bus_pic_data_process(business_df,all_df_pic)
business_res = bus_pic_data_process(business_res,res_df_pic)
business_drink = bus_pic_data_process(business_drink,drink_df_pic)


In [73]:
# business 均值和标准差的统计
def bus_result_mean(data):
    res = [
    round(len(data[data['stars']==1.0])/len(data),3),
    round(len(data[data['stars']==1.5])/len(data),3),
    round(len(data[data['stars']==2.0])/len(data),3),
    round(len(data[data['stars']==2.5])/len(data),3),
    round(len(data[data['stars']==3.0])/len(data),3),
    round(len(data[data['stars']==3.5])/len(data),3),
    round(len(data[data['stars']==4.0])/len(data),3),
    round(len(data[data['stars']==4.5])/len(data),3),
    round(len(data[data['stars']==5.0])/len(data),3),
    round(data['star_avg'].mean(),3),
    round(data['memory_score'].mean(),3),
    round(data['review_count'].mean(),3),
    round(data['photo_count'].mean(),3),
    round(data['categories_counts'].mean(),3),
    round(data['star_std'].mean(),3),
    round(data['contents_score_avg'].mean(),3),
    # round(data['useful_avg'].mean(),3),
    # round(data['funny_avg'].mean(),3),
    # round(data['cool_avg'].mean(),3),
    # round(data['person_count'].mean(), 3),
    round(data['beauty_score'].mean(),3),
    round(data['sharpness_measure'].mean(),3),
    round(data['average_hue'].mean(), 3),
    round(data['average_saturation'].mean(),3),
    round(data['average_value'].mean(),3),
    round(data['person_total_count_x'].mean(),3),
    round(len(data[data['person_exist']==1])/len(data),3),
    round(data['var'].mean(),3),
    round(data['food'].mean(),3),
    round(data['drink'].mean(),3),
    round(data['menu'].mean(),3),
    round(data['inside'].mean(),3),
    round(data['outside'].mean(),3)
    ]
    return res

def bus_result_std(data):
    res = [
    '-','-','-','-','-','-','-','-','-',
    round(data['star_avg'].std(),3),
    round(data['memory_score'].std(),3),
    round(data['review_count'].std(),3),
    round(data['photo_count'].std(),3),
    round(data['categories_counts'].std(),3),
    round(data['star_std'].std(),3),
    round(data['contents_score_avg'].std(),3),
    # round(data['useful_avg'].std(),3),
    # round(data['funny_avg'].std(),3),
    # round(data['cool_avg'].std(),3),
    # round(data['person_count'].std(), 3),
    round(data['beauty_score'].std(),3),
    round(data['sharpness_measure'].std(),3),
    round(data['average_hue'].std(), 3),
    round(data['average_saturation'].std(),3),
    round(data['average_value'].std(),3),
    round(data['person_total_count_x'].std(),3),
    '-',
    round(data['var'].std(),3),
    round(data['food'].std(),3),
    round(data['drink'].std(),3),
    round(data['menu'].std(),3),
    round(data['inside'].std(),3),
    round(data['outside'].std(),3)
    ]
    return res

In [74]:
# business 均值和标准差的统计结果
data = {
    "name":["1","1.5","2","2.5","3","3.5","4","4.5","5",
            "rating","memory_score","review_count","photo_count",
    "categories_counts","star_std","contents_score_avg",
    # "useful_avg","funny_avg","cool_avg","person_count",
    "beauty_score","sharpness_measure","h","s","v",
    'person_total_count','person_exist','var',
    'food','drink','menu','inside','outside'
    ],
    'All Busi mean': bus_result_mean(business_df),
    'All Busi std': bus_result_std(business_df),
    'All Res mean': bus_result_mean(business_res),
    'All Res std': bus_result_std(business_res),
    'Drink mean': bus_result_mean(business_drink),
    'Drink std': bus_result_std(business_drink)
}
# 创建DataFrame
df = pd.DataFrame(data)

# 显示DataFrame
df.head()

df.to_excel("data/output/study2_business_data_summary_yolo11.xlsx",index=False)
print("数据保存成功")

数据保存成功


In [75]:
# 模型前对数据进行标准化
scaler = StandardScaler()
business_df[['average_hue','average_saturation','average_value','sharpness_measure']] =  scaler.fit_transform(business_df[['average_hue','average_saturation','average_value','sharpness_measure']])
business_res[['average_hue','average_saturation','average_value','sharpness_measure']] =  scaler.fit_transform(business_res[['average_hue','average_saturation','average_value','sharpness_measure']])
business_drink[['average_hue','average_saturation','average_value','sharpness_measure']] =  scaler.fit_transform(business_drink[['average_hue','average_saturation','average_value','sharpness_measure']])

In [76]:
# # 保存数据用于做ivtest
business_df['review_group'] = pd.qcut(business_df['review_count'], q=[0, 2/3, 1], labels=['low', 'high'])
business_df.to_excel("data/output/study1_2_business_data.xlsx",index=False)
business_res['review_group'] = pd.qcut(business_res['review_count'], q=[0, 2/3, 1], labels=['low', 'high'])
business_res.to_excel("data/output/study1_2_res_data.xlsx",index=False) 
business_drink['review_group'] = pd.qcut(business_drink['review_count'], q=[0, 2/3, 1], labels=['low', 'high']) 
business_drink.to_excel("data/output/study1_2_drink_data.xlsx",index=False)


In [77]:
def w2sls_model_bus_res(data: pd.DataFrame):
    data = data.copy() 
    data['memory_score'] = data['memory_score'] - data['memory_score'].mean()
    data['review_group'] = pd.qcut(data['review_count'], q=[0, 2/3, 1], labels=['low', 'high'])
    data['review_high'] = (data['review_group'] == 'high').astype(int)
    data["memory_score_review_high"]= data['memory_score']*data['review_high']
    data["sharpness_measure_review_high"]= data['sharpness_measure']*data['review_high']
    data['log_photo_count'] = np.log(data['photo_count'])
    w2sls_model = IV2SLS(
    data["star_avg"], 
    sm.add_constant(data[['categories_counts', 
    'average_hue', 'average_saturation', 'average_value', 
    'food', 'drink', 'menu', 'inside', 
    'var', 'person_exist', 'beauty_score', 
    'review_high'
    ,'log_photo_count'
    ]]), 
    data[["memory_score","memory_score_review_high"]], 
    data[["sharpness_measure","sharpness_measure_review_high","person_total_count_x"]]
    # weights=data["review_count"]
    ).fit(cov_type='robust')
    return w2sls_model


In [78]:
# order模型结果
print('--------------------------------------------------1.all_business-----------------------------------------')
order_df = w2sls_model_bus_res(business_df)
print(order_df)
print('--------------------------------------------------2.restaurant--------------------------------------------------')
order_res = w2sls_model_bus_res(business_res)
print(order_res)


--------------------------------------------------1.all_business-----------------------------------------
                          IV-2SLS Estimation Summary                          
Dep. Variable:               star_avg   R-squared:                      0.1121
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1117
No. Observations:               26769   F-statistic:                    3791.8
Date:                Wed, Feb 18 2026   P-value (F-stat)                0.0000
Time:                        15:47:07   Distribution:                 chi2(15)
Cov. Estimator:                robust                                         
                                                                              
                                    Parameter Estimates                                     
                          Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------

In [79]:
# 模型预测结果对比图数据加工
from linearmodels.iv import IV2SLS
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


def fit_model_for_plot_2sls(df):
    """All Business / Restaurant 模型"""
    df2 = df.copy()
    mem_mean = df2['memory_score'].mean()
    df2['memory_score'] = df2['memory_score'] - df2['memory_score'].mean()
    df2['review_group'] = pd.qcut(df2['review_count'], q=[0, 2/3, 1], labels=['low', 'high'])
    df2['review_high'] = (df2['review_group'] == 'high').astype(int)
    df2['memory_score_review_high'] = df2['memory_score'] * df2['review_high']
    df2['sharpness_measure_review_high'] = df2['sharpness_measure'] * df2['review_high']
    df2['log_photo_count'] = np.log(df2['photo_count'])  #  加log变换

    exog = sm.add_constant(df2[['categories_counts',
                                'average_hue', 'average_saturation', 'average_value',
                                'food', 'drink', 'menu', 'inside',
                                'var', 'person_exist', 'beauty_score',
                                'review_high', 
                                'log_photo_count']])  # 改为log_photo_count
    endog = df2[['memory_score', 'memory_score_review_high']]
    instruments = df2[['sharpness_measure', 'sharpness_measure_review_high', 'person_total_count_x']]

    model = IV2SLS(df2['star_avg'], exog, endog, instruments).fit(cov_type='robust')
    return model, df2, mem_mean 


def fit_model_for_plot_2sls_drink(df):
    """Drink 模型"""
    df2 = df.copy()
    mem_mean = df2['memory_score'].mean()
    df2['memory_score'] = df2['memory_score'] - df2['memory_score'].mean()
    df2['review_group'] = pd.qcut(df2['review_count'], q=[0, 2/3, 1], labels=['low', 'high'])
    df2['review_high'] = (df2['review_group'] == 'high').astype(int)
    df2['memory_score_review_high'] = df2['memory_score'] * df2['review_high']
    df2['sharpness_measure_review_high'] = df2['sharpness_measure'] * df2['review_high']
    df2['person_total_count_review_high'] = df2['person_total_count_x'] * df2['review_high']
    df2['log_photo_count'] = np.log(df2['photo_count'])  #添加log变换

    exog = sm.add_constant(df2[['categories_counts', 'sharpness_measure',
                                'average_saturation', 'average_value',
                                'food', 'drink', 'menu', 'inside',
                                'var', 'person_exist', 'beauty_score',  
                                'review_high',
                                'log_photo_count']])  # 改为log_photo_count
    endog = df2[['memory_score', 'memory_score_review_high']]
    instruments = df2[['person_total_count_review_high', 'person_total_count_x', 'average_hue']]

    model = IV2SLS(df2['star_avg'], exog, endog, instruments).fit(cov_type='robust')
    return model, df2, mem_mean 


def make_pred_df_for_moderation(df, model, sample_label,mem_mean):
    """
    All Business / Restaurant 预测函数
    df: 对应样本的数据
    model: fit 出来的回归对象
    """
    dfc = df.copy()

    # memory 范围：5% ~ 95% 分位
    mem_min = dfc['memory_score'].quantile(0.05)
    mem_max = dfc['memory_score'].quantile(0.95)
    mem_grid = np.linspace(mem_min, mem_max, 200)

    control_vars = [
        'categories_counts',
        'average_hue', 'average_saturation', 'average_value',
        'food', 'drink', 'menu', 'inside',
        'var', 'person_exist', 'beauty_score'
    ]
    control_means = dfc[control_vars].mean()

    rows = []
    for rh in [0, 1]:  # 0 = emerging, 1 = established
        group_mean_rc = dfc.loc[dfc['review_high'] == rh, 'photo_count'].mean()
        for m in mem_grid:
            row = {}
            for v in control_vars:
                row[v] = control_means[v]
            row['review_high'] = rh
            row['log_photo_count'] = np.log(group_mean_rc)  # 改为log_photo_count
            row['memory_score'] = m
            row['memory_score_review_high'] = m * rh
            row['sample'] = sample_label
            rows.append(row)

    newdata = pd.DataFrame(rows)

    # 用模型系数计算预测值和 CI
    param_names = model.params.index.tolist()
    newdata = newdata.assign(const=1.0)
    for name in param_names:
        if name not in newdata.columns:
            newdata[name] = 0.0

    X = newdata[param_names].values
    beta = model.params.values
    cov = model.cov.loc[param_names, param_names].values

    y_hat = X @ beta
    var_hat = np.einsum('ij,jk,ik->i', X, cov, X)
    se_hat = np.sqrt(np.maximum(var_hat, 0))

    newdata['pred'] = y_hat
    newdata['ci_low'] = y_hat - 1.96 * se_hat
    newdata['ci_high'] = y_hat + 1.96 * se_hat

    newdata['memory_score'] = np.tile(mem_grid, 2)+mem_mean
    newdata['review_high'] = np.repeat([0, 1], len(mem_grid))
    newdata['status'] = np.where(newdata['review_high'] == 1,
                                 'Established (high visibility)',
                                 'Emerging (low visibility)')
    return newdata


def make_pred_df_for_moderation_drink(df, model, sample_label,mem_mean):
    """
    Drink 专用预测函数（control_vars 不同）
    """
    dfc = df.copy()

    mem_min = dfc['memory_score'].quantile(0.05)
    mem_max = dfc['memory_score'].quantile(0.95)
    mem_grid = np.linspace(mem_min, mem_max, 200)

    # Drink模型的control_vars不同
    control_vars = [
        'categories_counts', 
        'sharpness_measure',  # Drink用sharpness_measure替代average_hue
        'average_saturation', 'average_value',
        'food', 'drink', 'menu', 'inside',
        'var', 'person_exist', 'beauty_score'
    ]
    control_means = dfc[control_vars].mean()

    rows = []
    for rh in [0, 1]:
        group_mean_rc = dfc.loc[dfc['review_high'] == rh, 'photo_count'].mean()
        for m in mem_grid:
            row = {}
            for v in control_vars:
                row[v] = control_means[v]
            row['review_high'] = rh
            row['log_photo_count'] = np.log(group_mean_rc)  # 改为log_photo_count
            row['memory_score'] = m
            row['memory_score_review_high'] = m * rh
            row['sample'] = sample_label
            rows.append(row)

    newdata = pd.DataFrame(rows)

    param_names = model.params.index.tolist()
    newdata = newdata.assign(const=1.0)
    for name in param_names:
        if name not in newdata.columns:
            newdata[name] = 0.0

    X = newdata[param_names].values
    beta = model.params.values
    cov = model.cov.loc[param_names, param_names].values

    y_hat = X @ beta
    var_hat = np.einsum('ij,jk,ik->i', X, cov, X)
    se_hat = np.sqrt(np.maximum(var_hat, 0))

    newdata['pred'] = y_hat
    newdata['ci_low'] = y_hat - 1.96 * se_hat
    newdata['ci_high'] = y_hat + 1.96 * se_hat

    newdata['memory_score'] = np.tile(mem_grid, 2)+mem_mean
    newdata['review_high'] = np.repeat([0, 1], len(mem_grid))
    newdata['status'] = np.where(newdata['review_high'] == 1,
                                 'Established (high visibility)',
                                 'Emerging (low visibility)')
    return newdata


def plot_moderation(pred_df, title):
    fig, ax = plt.subplots(figsize=(6, 4))

    for rh, label in [(0, 'Emerging (low visibility)'),
                      (1, 'Established (high visibility)')]:
        tmp = pred_df[pred_df['review_high'] == rh].sort_values('memory_score')

        x = tmp['memory_score'].astype(float).values
        y = tmp['pred'].astype(float).values
        y1 = tmp['ci_low'].astype(float).values
        y2 = tmp['ci_high'].astype(float).values

        ax.plot(x, y, label=label)
        ax.fill_between(x, y1, y2, alpha=0.2)

    ax.set_xlabel('Image memorability (business level)')
    ax.set_ylabel('Predicted average rating')
    ax.set_title(title)
    ax.legend(title='Business status')
    fig.tight_layout()
    return fig


# ============== 调用代码 ==============
# 1）All businesses
model_bus, bus_clean, mem_mean = fit_model_for_plot_2sls(business_df)
bus_pred = make_pred_df_for_moderation(bus_clean, model_bus, "All businesses", mem_mean)

# 2）Restaurants
model_res, res_clean, mem_mean = fit_model_for_plot_2sls(business_res)
res_pred = make_pred_df_for_moderation(res_clean, model_res, "Restaurants", mem_mean)

# 3）Drinks-使用专用函数
model_drink, drink_clean, mem_mean = fit_model_for_plot_2sls_drink(business_drink)
drink_pred = make_pred_df_for_moderation_drink(drink_clean, model_drink, "Drinks", mem_mean)

# ============== 保存数据为 CSV ==============
all_pred = pd.concat([bus_pred, res_pred, drink_pred], ignore_index=True)
all_pred.to_csv("data/output/plot_data_moderation_all.csv", index=False)

print("数据已保存！")
print(all_pred[['sample', 'memory_score', 'review_high', 'status', 'pred', 'ci_low', 'ci_high']].head(10))

数据已保存！
           sample  memory_score  review_high                     status  \
0  All businesses      0.618850            0  Emerging (low visibility)   
1  All businesses      0.620182            0  Emerging (low visibility)   
2  All businesses      0.621515            0  Emerging (low visibility)   
3  All businesses      0.622847            0  Emerging (low visibility)   
4  All businesses      0.624180            0  Emerging (low visibility)   
5  All businesses      0.625512            0  Emerging (low visibility)   
6  All businesses      0.626844            0  Emerging (low visibility)   
7  All businesses      0.628177            0  Emerging (low visibility)   
8  All businesses      0.629509            0  Emerging (low visibility)   
9  All businesses      0.630842            0  Emerging (low visibility)   

       pred    ci_low   ci_high  
0  3.707187  3.577470  3.836905  
1  3.705933  3.577374  3.834492  
2  3.704679  3.577277  3.832080  
3  3.703424  3.577180  3.829669

In [80]:
def w2sls_model_drink(data: pd.DataFrame):
    data = data.copy() 
    data['memory_score'] = data['memory_score'] - data['memory_score'].mean()
    data['review_group'] = pd.qcut(data['review_count'], q=[0, 2/3, 1], labels=['low', 'high'])
    data['review_high'] = (data['review_group'] == 'high').astype(int)
    data["memory_score_review_high"]= data['memory_score']*data['review_high']
    data["sharpness_measure_review_high"]= data['sharpness_measure']*data['review_high']
    data["person_total_count_review_high"]= data['person_total_count_x']*data['review_high']
    data['log_photo_count'] = np.log(data['photo_count'])
    w2sls_model = IV2SLS(
    data["star_avg"],
    sm.add_constant(data[['categories_counts', "sharpness_measure",
    'average_saturation', 'average_value', 
    'food', 'drink', 'menu', 'inside', 
    'var', 'person_exist', 'beauty_score', 
    'review_high'
    ,'log_photo_count']]), 
    data[["memory_score","memory_score_review_high"]], 
    data[["person_total_count_review_high","person_total_count_x",'average_hue']]
    # weights=data["review_count"]
    ).fit(cov_type='robust')
    return w2sls_model


print('--------------------------------------------------3.drink--------------------------------------------------')
order_drink = w2sls_model_drink(business_drink)
print(order_drink)

--------------------------------------------------3.drink--------------------------------------------------
                          IV-2SLS Estimation Summary                          
Dep. Variable:               star_avg   R-squared:                      0.1294
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1281
No. Observations:                9751   F-statistic:                    1229.0
Date:                Wed, Feb 18 2026   P-value (F-stat)                0.0000
Time:                        15:47:07   Distribution:                 chi2(15)
Cov. Estimator:                robust                                         
                                                                              
                                    Parameter Estimates                                     
                          Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------

In [81]:
def data_split_new(df):
    df['review_group'] = pd.qcut(df['review_count'], q=[0, 2/3, 1], labels=['low', 'high'])
    df_high = df[df['review_group']=='high']
    df_low = df[df['review_group']=='low']
    return df_high,df_low

business_df_high,business_df_low = data_split_new(business_df)
business_res_high,business_res_low = data_split_new(business_res)
business_drink_high,business_drink_low = data_split_new(business_drink)


In [82]:
print("----------------------------高低评论数量 情感的倾向均值-------------------------")
# business
def sentiment_analysis(business_df_high,business_df_low,file_name):
    from scipy import stats
    print("high",round(business_df_high['contents_score_avg'].mean(),3),
    "low",round(business_df_low['contents_score_avg'].mean(),3))
    print("memory score")
    print("high",round(business_df_high['memory_score'].mean(),3),
    "low",round(business_df_low['memory_score'].mean(),3))
    # ANOVA
    t_stat, p_val = stats.f_oneway(business_df_high['contents_score_avg'],
                                    business_df_low['contents_score_avg'])
    print(f"ANOVA结果: F = {t_stat:.3f}, p = {p_val:.4f}")
    t_stat, p_val = stats.f_oneway(business_df_high['memory_score'],
                                    business_df_low['memory_score'])
    print(f"ANOVA结果: F = {t_stat:.3f}, p = {p_val:.4f}")
    # =====ANOVA结果显著，可以做事后检验（Tukey HSD）=====
    import statsmodels.api as sm
    from statsmodels.stats.multicomp import pairwise_tukeyhsd

    # 拼接成一个DataFrame
    df = pd.DataFrame({'sentiment_score': pd.concat([business_df_high['contents_score_avg'], 
                                        business_df_low['contents_score_avg']]),
        "group": (["High"]*len(business_df_high) +
                ["Low"]*len(business_df_low))
    })

    tukey = pairwise_tukeyhsd(endog=df['sentiment_score'], groups=df["group"], alpha=0.05)
    print(tukey)

    df.to_excel("data/output/{}.xlsx".format(file_name),index=False)

    # 拼接成一个DataFrame
    df = pd.DataFrame({'sentiment_score': pd.concat([business_df_high['memory_score'], 
                                        business_df_low['memory_score']]),
        "group": (["High"]*len(business_df_high) +
                ["Low"]*len(business_df_low))
    })
    tukey = pairwise_tukeyhsd(endog=df['sentiment_score'], groups=df["group"], alpha=0.05)
    print(tukey)


print("sentiment analysis")
print("--------------------------------------------------1.all_business--------------------------------------------------")
sentiment_analysis(business_df_high,business_df_low,"all_business_sentiment")
print("--------------------------------------------------2.restaurant--------------------------------------------------")
sentiment_analysis(business_res_high,business_res_low,"restaurant_sentiment")
print("--------------------------------------------------3.drink--------------------------------------------------")
sentiment_analysis(business_drink_high,business_drink_low,"drink_sentiment")


----------------------------高低评论数量 情感的倾向均值-------------------------
sentiment analysis
--------------------------------------------------1.all_business--------------------------------------------------
high 0.702 low 0.536
memory score
high 0.736 low 0.771
ANOVA结果: F = 2321.246, p = 0.0000
ANOVA结果: F = 1164.557, p = 0.0000
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj lower   upper  reject
--------------------------------------------------
  High    Low  -0.1662   0.0 -0.173 -0.1595   True
--------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj lower  upper  reject
-------------------------------------------------
  High    Low   0.0352   0.0 0.0332 0.0372   True
-------------------------------------------------
--------------------------------------------------2.restaurant--------------------------------------------------
high 0.707 low 0.537
memory score
high 0.734 low 0.765